In [1]:
library('SelectSim')
library('tidyverse')
library('tictoc')
library("discover")

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] discover_0.9.4    tictoc_1.2        lubridate_1.9.3   forcats_1.0.0    
 [5] stringr_1.5.1     dplyr_1.1.4       purrr_1.0.2       readr_2.1.5      
 [9] tidyr_1.3.0       tibble_3.2.1      ggplot2_3.4.4     tidyverse_2.0.0  

## SelectSim run

In [3]:
filter_run_data <- function(run_data,gene_order,sample_order){
    final_run_data<-run_data
    final_run_data$M$M$missense<-run_data$M$M$missense[gene_order,sample_order]
    final_run_data$M$M$truncating<-run_data$M$M$truncating[gene_order,sample_order]
    final_run_data$alteration.class<- run_data$alteration.class[gene_order]
    final_run_data$sample.class<-run_data$sample.class[sample_order]
    final_run_data$M$tmb$missense<-run_data$M$tmb$missense[sample_order,]
    final_run_data$M$tmb$truncating<-run_data$M$tmb$truncating[sample_order,]
    return(final_run_data)
}

In [4]:
run_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_tcga_run_data.rds')

In [5]:
sample_covariates<-run_data$sample.class

In [6]:
luad_samples <- names(sample_covariates[(which(sample_covariates %in% c('LUAD')))])

In [7]:
luad_run_data<- filter_run_data(run_data = run_data,gene_order = names(run_data$alteration.class),sample_order = luad_samples)

In [8]:
str(luad_run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:502] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  .. ..$ truncating: num [1:396, 1:502] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	502 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  .. .. ..$ mutation: num [1:502] 163 253 270 1328 100 ...
  .. ..$ truncating:'data.frame':	502 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-0

In [9]:
gam<-luad_run_data$M$M[[1]]+luad_run_data$M$M[[2]][rownames(luad_run_data$M$M[[1]]),colnames(luad_run_data$M$M[[1]])]
gam[gam>1]<-1

In [10]:
tmb<- run_data$M$tmb$missense
tmb$mutation<-tmb$mutation+run_data$M$tmb$truncating$mutation

In [11]:
head(tmb)

,sample,mutation
,<chr>,<dbl>
TCGA-02-0047-01,TCGA-02-0047-01,63
TCGA-02-0055-01,TCGA-02-0055-01,50
TCGA-AB-2805-03,TCGA-AB-2805-03,57
TCGA-AB-2806-03,TCGA-AB-2806-03,507
TCGA-AB-2807-03,TCGA-AB-2807-03,261
TCGA-AB-2808-03,TCGA-AB-2808-03,771


In [12]:
dim(gam)

[1] 396 502

In [13]:
data <-list('M'=list('mut'=gam),
            'tmb'=list('mut'=tmb))

alteration_covariates <- luad_run_data$alteration.class
sample_covariates<-luad_run_data$sample.class[luad_samples]

In [14]:
run_data <- list('M'=data,'sample.class' = sample_covariates,'alteration.class' = alteration_covariates)

In [15]:
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 1
  .. ..$ mut: num [1:396, 1:502] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  ..$ tmb:List of 1
  .. ..$ mut:'data.frame':	9082 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  .. .. ..$ mutation: num [1:9082] 63 50 57 507 261 771 17 11 20 10 ...
 $ sample.class    : Named chr [1:502] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
 $ alteration.class: Named chr [1:396] "MUT" "MUT" "MUT" "MUT" ...
  ..- attr(*, "names")= chr [1:396] "AKT1" "ALK" "APC" "AR" ...


In [ ]:
test<-SelectSim::selectX(  M = run_data$M,
                sample.class = run_data$sample.class,
                alteration.class = run_data$alteration.class,
                n.cores = 20,
                min.freq = (0.001*502),
                n.permut = 1000,
                lambda = 0.3,
                tao = 1,
                save.object = FALSE,
                verbose = FALSE,
                estimate_pairwise = FALSE,
                maxFDR = 0.25)

In [16]:
dim(test$result)

[1] 36046    22

In [17]:
test$result %>% filter(FDR) %>% dplyr::count(type)

type,n
<chr>,<int>
CO,12
ME,48


In [18]:
selectX_obj <- test$obj
selectX_result <- test$result

In [19]:
saveRDS(selectX_result,file='tcga_luad_result_v4.rds')
saveRDS(selectX_obj,file='tcga_luad_obj_v4.rds')

In [20]:
dim(gam)

[1] 396 502

# Discover Run

In [16]:
global_gam <- gam

In [17]:
head(global_gam)

,TCGA-05-4244-01,TCGA-05-4249-01,TCGA-05-4250-01,TCGA-05-4382-01,TCGA-05-4384-01,TCGA-05-4389-01,TCGA-05-4390-01,TCGA-05-4395-01,TCGA-05-4396-01,TCGA-05-4397-01,⋯,TCGA-MP-A4T6-01,TCGA-MP-A4TC-01,TCGA-MP-A4TH-01,TCGA-MP-A4TJ-01,TCGA-MP-A5C7-01,TCGA-NJ-A4YF-01,TCGA-NJ-A55A-01,TCGA-NJ-A7XG-01,TCGA-O1-A52J-01,TCGA-S2-AA1A-01
AKT1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ALK,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
APC,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,1,0,0,0,0,0
AR,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ATM,0,0,0,1,0,0,1,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0
BAP1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [18]:
gam_discover <-discover.matrix(global_gam)

In [19]:
head(gam_discover$bg)

,TCGA-05-4244-01,TCGA-05-4249-01,TCGA-05-4250-01,TCGA-05-4382-01,TCGA-05-4384-01,TCGA-05-4389-01,TCGA-05-4390-01,TCGA-05-4395-01,TCGA-05-4396-01,TCGA-05-4397-01,⋯,TCGA-MP-A4T6-01,TCGA-MP-A4TC-01,TCGA-MP-A4TH-01,TCGA-MP-A4TJ-01,TCGA-MP-A5C7-01,TCGA-NJ-A4YF-01,TCGA-NJ-A55A-01,TCGA-NJ-A7XG-01,TCGA-O1-A52J-01,TCGA-S2-AA1A-01
AKT1,0.005638681,0.005638681,0.005638681,0.02965670,0.005638681,0.003549741,0.010278638,0.005638681,0.012783997,0.010278638,⋯,0.001657051,0.005638681,1.513182e-15,0.001657051,0.003549741,0.005638681,0.001657051,1.513182e-15,0.001657051,1.513182e-15
ALK,0.011321828,0.011321828,0.011321828,0.05813206,0.011321828,0.007142598,0.020541723,0.011321828,0.025484237,0.020541723,⋯,0.003340651,0.011321828,3.055761e-15,0.003340651,0.007142598,0.011321828,0.003340651,3.055761e-15,0.003340651,3.055761e-15
APC,0.018967366,0.018967366,0.018967366,0.09437089,0.018967366,0.012000191,0.034198023,0.018967366,0.042284506,0.034198023,⋯,0.005627241,0.018967366,5.159187e-15,0.005627241,0.012000191,0.018967366,0.005627241,5.159187e-15,0.005627241,5.159187e-15
AR,0.003754147,0.003754147,0.003754147,0.01990567,0.003754147,0.002361705,0.006854048,0.003754147,0.008531879,0.006854048,⋯,0.001101764,0.003754147,1.005547e-15,0.001101764,0.002361705,0.003754147,0.001101764,1.005547e-15,0.001101764,1.005547e-15
ATM,0.042355560,0.042355560,0.042355560,0.19249357,0.042355560,0.027034074,0.074932289,0.042355560,0.091735925,0.074932289,⋯,0.012780347,0.042355560,1.180222e-14,0.012780347,0.027034074,0.042355560,0.012780347,1.180222e-14,0.012780347,1.180222e-14
BAP1,0.007528155,0.007528155,0.007528155,0.03927651,0.007528155,0.004742568,0.013701499,0.007528155,0.017026807,0.013701499,⋯,0.002215287,0.007528155,2.024082e-15,0.002215287,0.004742568,0.007528155,0.002215287,2.024082e-15,0.002215287,2.024082e-15


In [21]:
length(names(which(rowSums(gam)>0.5)))

[1] 269

In [22]:
substet <-  rowSums(global_gam)>0.5

In [23]:
result.mutex <- pairwise.discover.test(gam_discover[substet, ],fdr.method='DBH',alternative='less')
result.co <- pairwise.discover.test(gam_discover[substet, ],fdr.method='DBH',alternative='greater')
ME_discover <- as.data.frame(result.mutex,q.threshold=0.25) 
CO_discover <- as.data.frame(result.co,q.threshold=0.25) 
discover_result <- rbind(ME_discover,CO_discover)
discover_result$type <- c(rep('ME',nrow(ME_discover)),rep('CO',nrow(CO_discover)))

In [24]:
head(discover_result)

,gene1,gene2,p.value,q.value,type
,<chr>,<chr>,<dbl>,<dbl>,<chr>
9,KRAS,EGFR,2.308348e-11,6.193387e-11,ME
24,TP53,KRAS,4.014858e-06,1.421706e-05,ME
34,TP53,STK11,3.049509e-05,9.946465e-05,ME
4,KRAS,BRAF,4.376552e-05,1.184953e-04,ME
12,STK11,EGFR,5.302269e-05,1.242593e-04,ME
23,SMARCA4,KRAS,2.417597e-03,7.493644e-03,ME


In [25]:
result.mutex

Pairwise DISCOVER mutual exclusivity test
alternative hypothesis: observed overlap is less than expected by chance
FDR estimation method: discrete Benjamini-Hochberg

number of pairs tested: 36046
proportion of true null hypotheses: 0.9973936
number of significant pairs at a maximum FDR of 0.01 : 7

In [26]:
result.co

Pairwise DISCOVER co-occurrence test
alternative hypothesis: observed overlap is greater than expected by chance
FDR estimation method: discrete Benjamini-Hochberg

number of pairs tested: 36046
proportion of true null hypotheses: 1
number of significant pairs at a maximum FDR of 0.01 : 0

In [27]:
dim(discover_result)

[1] 38  5

In [28]:
head(discover_result)

,gene1,gene2,p.value,q.value,type
,<chr>,<chr>,<dbl>,<dbl>,<chr>
9,KRAS,EGFR,2.308348e-11,6.193387e-11,ME
24,TP53,KRAS,4.014858e-06,1.421706e-05,ME
34,TP53,STK11,3.049509e-05,9.946465e-05,ME
4,KRAS,BRAF,4.376552e-05,1.184953e-04,ME
12,STK11,EGFR,5.302269e-05,1.242593e-04,ME
23,SMARCA4,KRAS,2.417597e-03,7.493644e-03,ME


In [36]:
saveRDS(gam_discover,file='discover_luad_obj_v4.rds')

# Discover +

In [50]:
str(selectX_obj)

List of 5
 $ al  :List of 4
  ..$ am         :List of 2
  .. ..$ full: num [1:269, 1:502] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:269] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  .. ..$ mut : num [1:269, 1:502] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:269] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  ..$ alterations:List of 1
  .. ..$ alteration.class: Named chr [1:396] "MUT" "MUT" "MUT" "MUT" ...
  .. .. ..- attr(*, "names")= chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  ..$ samples    :List of 1
  .. ..$ sample.class: Named chr [1:502] "LUAD" "LUAD" "LUAD" "LUAD" ...
  .. .. ..- attr(*, "names")= chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  ..$ tmb        :List of 2
  

In [51]:
str(gam_discover$bg)

 num [1:396, 1:502] 0.00564 0.01132 0.01897 0.00375 0.04236 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...


In [52]:
head(gam_discover$bg)

,TCGA-05-4244-01,TCGA-05-4249-01,TCGA-05-4250-01,TCGA-05-4382-01,TCGA-05-4384-01,TCGA-05-4389-01,TCGA-05-4390-01,TCGA-05-4395-01,TCGA-05-4396-01,TCGA-05-4397-01,⋯,TCGA-MP-A4T6-01,TCGA-MP-A4TC-01,TCGA-MP-A4TH-01,TCGA-MP-A4TJ-01,TCGA-MP-A5C7-01,TCGA-NJ-A4YF-01,TCGA-NJ-A55A-01,TCGA-NJ-A7XG-01,TCGA-O1-A52J-01,TCGA-S2-AA1A-01
AKT1,0.005638681,0.005638681,0.005638681,0.02965670,0.005638681,0.003549741,0.010278638,0.005638681,0.012783997,0.010278638,⋯,0.001657051,0.005638681,1.513182e-15,0.001657051,0.003549741,0.005638681,0.001657051,1.513182e-15,0.001657051,1.513182e-15
ALK,0.011321828,0.011321828,0.011321828,0.05813206,0.011321828,0.007142598,0.020541723,0.011321828,0.025484237,0.020541723,⋯,0.003340651,0.011321828,3.055761e-15,0.003340651,0.007142598,0.011321828,0.003340651,3.055761e-15,0.003340651,3.055761e-15
APC,0.018967366,0.018967366,0.018967366,0.09437089,0.018967366,0.012000191,0.034198023,0.018967366,0.042284506,0.034198023,⋯,0.005627241,0.018967366,5.159187e-15,0.005627241,0.012000191,0.018967366,0.005627241,5.159187e-15,0.005627241,5.159187e-15
AR,0.003754147,0.003754147,0.003754147,0.01990567,0.003754147,0.002361705,0.006854048,0.003754147,0.008531879,0.006854048,⋯,0.001101764,0.003754147,1.005547e-15,0.001101764,0.002361705,0.003754147,0.001101764,1.005547e-15,0.001101764,1.005547e-15
ATM,0.042355560,0.042355560,0.042355560,0.19249357,0.042355560,0.027034074,0.074932289,0.042355560,0.091735925,0.074932289,⋯,0.012780347,0.042355560,1.180222e-14,0.012780347,0.027034074,0.042355560,0.012780347,1.180222e-14,0.012780347,1.180222e-14
BAP1,0.007528155,0.007528155,0.007528155,0.03927651,0.007528155,0.004742568,0.013701499,0.007528155,0.017026807,0.013701499,⋯,0.002215287,0.007528155,2.024082e-15,0.002215287,0.004742568,0.007528155,0.002215287,2.024082e-15,0.002215287,2.024082e-15


In [53]:
discover_selectX_obj <-selectX_obj

In [54]:
discover_selectX_obj$T$templeate.obj$mut$LUAD <- gam_discover$bg

In [55]:
discover_selectX_obj$T$temp_mat$mut <- gam_discover$bg

In [56]:
sim <- SelectX::null_model_parallel(discover_selectX_obj$al,
                                       discover_selectX_obj$T$temp_mat,
                                       discover_selectX_obj$W$W,
                                       1,
                                       1000)

In [57]:
length(sim)

[1] 1000

In [58]:
discover_selectX_obj$null <- sim
discover_selectX_obj$nSim <- length(sim)

In [59]:
test_fun <- function(obj){    
    al<-obj$al
    als = SelectX::al.stats(obj) # Univariate stats
    alp = SelectX::al.pairwise.alteration.stats(obj, als, do.blocks=FALSE)
    als[['alteration.pairwise']] = alp
    als[['alteration.pairwise']]$sample.blocks = NULL
    for(i in names(als$sample.blocks)) als$sample.blocks[[i]][['alteration.pairwise']] = alp$sample.blocks[[i]]
    obs.co = as.matrix(als$alteration.pairwise$overlap)
    wobs.co = as.matrix(als$alteration.pairwise$w_overlap)
    robs.co<-SelectX::r.am.pairwise.alteration.overlap(null = obj$null,
                              n.permut = obj$nSim,
                              n.cores = 1)
    wrobs.co<-SelectX::w.r.am.pairwise.alteration.overlap(null = obj$null,
                                 W= obj$W$W,
                                 n.permut = obj$nSim,
                                 n.cores = 1)

    selectX_result <- interaction.table(al,
                                    als,
                                    obs.co,
                                    wobs.co,
                                    robs.co,
                                    wrobs.co,
                                    null=obj$null,
                                    maxFDR=0.25,
                                    n.cores=1,
                                    estimate_pairwise=FALSE,
                                    n.permut=obj$nSim)
}

In [60]:
disocver_selectX_result <- test_fun(discover_selectX_obj)

In [61]:
disocver_selectX_result %>% filter(nFDR2<=0.25) %>% dplyr::count(type)

type,n
<chr>,<int>
CO,12
ME,14


In [62]:
disocver_selectX_result<-disocver_selectX_result %>% mutate(id=case_when(SFE_1>SFE_2 ~ paste(SFE_2,SFE_1,sep="_"),
                                SFE_1<SFE_2 ~ paste(SFE_1,SFE_2,sep="_")
                               )
                  )

In [64]:
dim(discover_result)
dim(disocver_selectX_result)

[1] 38  6

[1] 36046    23

In [65]:
disocver_selectX_result %>% filter(FDR) %>% arrange(desc(overlap)) %>% dplyr::select(SFE_1,SFE_2,overlap,w_overlap,r_overlap,w_r_overlap,nES,cum_freq,type,,FDR)

,SFE_1,SFE_2,overlap,w_overlap,r_overlap,w_r_overlap,nES,cum_freq,type,FDR
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>
KRAS - TP53,KRAS,TP53,49,35.8760174,67.622,55.6307165,-11.382077,375,ME,TRUE
EGFR - TP53,EGFR,TP53,32,29.3335097,25.861,19.1079805,5.637006,278,CO,TRUE
KRAS - STK11,KRAS,STK11,28,25.9230769,18.111,14.9469632,6.066377,213,CO,TRUE
NF1 - TP53,NF1,TP53,22,17.1417004,14.408,11.4415631,2.704932,253,CO,TRUE
KRAS - RBM10,KRAS,RBM10,16,13.9725275,8.899,7.3077025,3.517927,183,CO,TRUE
RB1 - TP53,RB1,TP53,15,12.0045547,8.546,6.2311220,3.130294,240,CO,TRUE
STK11 - TP53,STK11,TP53,13,9.5456386,26.083,21.3880359,-6.606724,280,ME,TRUE
ATM - KRAS,ATM,KRAS,13,10.7706044,6.846,5.0670045,3.093742,176,CO,TRUE
KRAS - KEAP1,KRAS,KEAP1,13,12.5384615,10.122,8.2926058,1.769983,187,CO,TRUE


In [78]:
disover_selectX <- list('obj'=discover_selectX_obj,'result'=disocver_selectX_result)

In [79]:
saveRDS(disover_selectX,file='discover_with_selectX_luad_v4.rds')